# FCC-Fractionator

References:

a. Santander (2022) An open source fluid catalytic cracker - fractionator model to support the development and benchmarking of process control, machine learning and operation strategies [Computers and Chemical Engineering]  
b. Santander (2023) Deep Learning Model Predictive Control Frameworks: Application to a Fluid Catalytic Cracker-Fractionator Process [I&EC research]  
c. Santander (2023) Integrated Production Planning and Model Predictive Control of a Fluidized Bed Catalytic Cracking-Fractionator Unit [I&EC research]  
d. Kumar (2023) Fluid Catalytic Cracking Unit Dataset for Process Monitoring Evaluation [github](https://github.com/ML-PSE/Fluid-Catalytic-Cracking-Unit-Dataset-for-Process-Monitoring-Evaluation)

In [1]:
import os
import time

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.preprocessing as skp 
from tqdm.notebook import tqdm
import torch
from torch.func import functional_call

from utils import *
from utils.fcc import *
from utils.optimizers import maskedSGD, GEKF, maskedAdam
from utils.modeling import *

results_dir = os.path.join("results", "fcc")
os.makedirs(results_dir, exist_ok=True)

%config InlineBackend.figure_format = "png"
sns.set_context("talk")

MEASUREMENT_NOISE_LEVEL = 0.0025  # +/- of the mean value of each variable

## Normal Process Data
 import & visualize

In [4]:
# Normal Operating Conditions FCC Data
if 0:
    df_stable = pd.read_csv(NOC_STABLE_PATH, names=COLUMNS)
    df_varying = pd.read_csv(NOC_VARYING_PATH, names=COLUMNS)
    df = pd.concat([df_stable, df_varying], ignore_index=True)

    dfplot = df[sorted_columns]
    data = default_rng.standard_normal(dfplot.shape) * MEASUREMENT_NOISE_LEVEL/3 * dfplot.values.mean(axis=0) + dfplot.values
    dfplot = pd.DataFrame(data, columns=sorted_columns)
    
    fig, axs = graph_subplots(dfplot)

    # fig.savefig(os.path.join(results_dir, "NOC_data.png"), bbox_inches="tight", dpi=500)
    # fig.savefig(os.path.join(results_dir, "NOC_data.pdf"), bbox_inches="tight", dpi=1_000)
    # fig.savefig(os.path.join(results_dir, "NOC_data.svg"), bbox_inches="tight", dpi=500)
    plt.close(fig)
    
    scaler = skp.RobustScaler()
    dfplot_scaled = pd.DataFrame(scaler.fit_transform(dfplot), columns=sorted_columns)
    
    fig, axs = graph_subplots(dfplot_scaled)
    # fig.savefig(os.path.join(results_dir, "NOC_data_scaled.png"), bbox_inches="tight", dpi=500)
    # fig.savefig(os.path.join(results_dir, "NOC_data_scaled.pdf"), bbox_inches="tight", dpi=1_000)
    # fig.savefig(os.path.join(results_dir, "NOC_data_scaled.svg"), bbox_inches="tight", dpi=500)    

faulty operation data

In [5]:
# Faulty Operating Conditions FCC Data
if 0:
    sorted_columns = sorted(X_COLUMNS_NAMES) + sorted(U_COLUMNS_NAMES)

    df_stable = pd.read_csv(NOC_STABLE_PATH, names=COLUMNS)
    
    for k,v in FAULTY_CONDITIONS.items():
        df_faulty = pd.read_csv(v["path"], names=COLUMNS)
        df = pd.concat([df_stable, df_faulty], ignore_index=True)

        dfplot = df[sorted_columns]
        
        # add noise to the data
        data = default_rng.standard_normal(dfplot.shape) * MEASUREMENT_NOISE_LEVEL/3 * dfplot.values.mean(axis=0) + dfplot.values
        dfplot = pd.DataFrame(data, columns=sorted_columns)

        fig, axs = graph_subplots(dfplot)

        fig.savefig(os.path.join(results_dir, f"FOC_{k}_data.png"), bbox_inches="tight", dpi=500)
        fig.savefig(os.path.join(results_dir, f"FOC_{k}_data.pdf"), bbox_inches="tight", dpi=1_000)
        fig.savefig(os.path.join(results_dir, f"FOC_{k}_data.svg"), bbox_inches="tight", dpi=500)
        
        dfplot_scaled = pd.DataFrame(scaler.transform(dfplot), columns=sorted_columns)
        
        fig, axs = graph_subplots(dfplot_scaled)
        
        fig.savefig(os.path.join(results_dir, f"FOC_{k}_data_scaled.png"), bbox_inches="tight", dpi=500)
        fig.savefig(os.path.join(results_dir, f"FOC_{k}_data_scaled.pdf"), bbox_inches="tight", dpi=1_000)
        fig.savefig(os.path.join(results_dir, f"FOC_{k}_data_scaled.svg"), bbox_inches="tight", dpi=500)

training, validation, and test datasets

Add noise  
Type K thermocouples have are accurate to 2.2C or +/- 0.75%, whichever is greater. Assume this is true of all sensors: that they are within 1% in a Gaussian distribution. Also assume that this is incorporated in 3 standard deviations from the mean value.

In [6]:
# model = VanillaRNN(state_dim=len(X_COLUMNS), input_dim=len(U_COLUMNS), hidden_size=64)
# opt = torch.optim.Adam(model.parameters(), lr=1e-3)
# lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(opt, T_0=50, T_mult=1)

# lrs = []
# for e in range(200):
#     lrs.append(opt.param_groups[0]["lr"])
#     lr_scheduler.step()
    
# fig, ax = plt.subplots(figsize=(5, 3), dpi=300)

# ax.plot(lrs)
# ax.set_ylabel("Learning Rate")
# ax.set_xlabel("Epoch")

In [7]:
# import data
df_stable = pd.read_csv(NOC_STABLE_PATH, names=COLUMNS)
df_varying = pd.read_csv(NOC_VARYING_PATH, names=COLUMNS)
df = pd.concat([df_stable, df_varying], ignore_index=True)
# split and scale data
# 7 days train (2 stable, 5 varying) ~78%
# 1 day val ~11%
# 1 day test ~11%
train_val_idx = -2880
val_test_idx = -1440
prediction_horizon = 60
batch_size = 32

if MEASUREMENT_NOISE_LEVEL:
    noise_std = MEASUREMENT_NOISE_LEVEL / 3
    # Add Gaussian noise with st dev of 1/3% of the mean value of each variable
    data = default_rng.standard_normal(df.shape) * MEASUREMENT_NOISE_LEVEL/3 * df.values.mean(axis=0) + df.values

    df = pd.DataFrame(data, columns=df.columns)

# X_scaler = skp.QuantileTransformer(output_distribution='normal')
# U_scaler = skp.QuantileTransformer(output_distribution='normal')
# X_scaler = skp.RobustScaler(quantile_range=(10, 90))
# U_scaler = skp.RobustScaler(quantile_range=(10, 90))
# X_scaler = skp.MaxAbsScaler()
# U_scaler = skp.MaxAbsScaler()
# X_scaler = StandardScaler()
# U_scaler = StandardScaler()
X_scaler = skp.RobustScaler()
U_scaler = skp.RobustScaler()

train_dataset = FCUDataset(df, 
    prediction_horizon=prediction_horizon, 
    context_length=1, 
    train=True, 
    X_scaler=X_scaler, 
    U_scaler=U_scaler, 
    begin_split_index=0, 
    end_split_index=train_val_idx)
val_dataset = FCUDataset(df,
    prediction_horizon=prediction_horizon, 
    context_length=1, 
    train=False, 
    X_scaler=X_scaler, 
    U_scaler=U_scaler, 
    begin_split_index=train_val_idx, 
    end_split_index=val_test_idx)
test_dataset = FCUDataset(df,
    prediction_horizon=prediction_horizon, 
    context_length=1, 
    train=False, 
    X_scaler=X_scaler, 
    U_scaler=U_scaler, 
    begin_split_index=val_test_idx)

train_dl = DataLoader(train_dataset, batch_size=128, shuffle=True, generator=torch.Generator(device=device))
val_dl = DataLoader(val_dataset, batch_size=128, shuffle=False, generator=torch.Generator(device=device))
test_dl = DataLoader(test_dataset, batch_size=128, shuffle=False, generator=torch.Generator(device=device))

### Train models

Vanilla RNN - Adam

train loss: 0.1663  
val loss: 0.1709  

RKRNN - Adam

Train loss: 0.1681  
Val Loss: 0.1724  

## Updating

get gradients on validation dataset

In [10]:
logfilename = os.path.join(results_dir, "Faulty Conditions", "FC_GEKFv2.h5")
logger = h5_logger(logfilename, existing=True)
rk_trained_weights_path = os.path.join(results_dir, "training", "RkRNN_NOC.pth")

if not check_if_in_h5(logfilename, "training"):
    val_dl_individual = DataLoader(val_dataset, batch_size=1, shuffle=False, generator=torch.Generator(device=device))

    m = Exogenous_RkRNN(state_dim=len(X_COLUMNS), input_dim=len(U_COLUMNS), hidden_size=64)
    m.load_state_dict(torch.load(rk_trained_weights_path, map_location=device))
    opt = maskedSGD(m.parameters(), lr=1e-3)
    loss_fn = torch.nn.MSELoss()
    logger = h5_logger(logfilename, existing=True)


    pbar = tqdm(val_dl_individual)
    for data in pbar:
        opt.zero_grad()
        X0, U, X1 = data["X0"], data["U"], data["X1"]
        xp = m(X0, U)
        loss = loss_fn(xp, X1)
        loss.backward()
        grads = opt._get_flat_grads()
        logger.log_dict({
            "training/X0": X0.detach().cpu().numpy(),
            "training/U": U.detach().cpu().numpy(),
            "training/Y": X1.detach().cpu().numpy(),
            "training/YP": xp.detach().cpu().numpy(),
            "training/mse_scaled": np.array([loss.item()]),
            "training/grads": grads.detach().cpu().numpy()
        })

if check_if_in_h5(logfilename, "training/abs_grad_values"):
    grad_values = logger.get_dataset("training/abs_grad_values")[0]
else:
    grads = logger.get_dataset("training/grads")
    abs_grads = np.abs(grads).reshape(-1)
    quantiles = np.linspace(0, 1, 101)
    grad_values = [np.quantile(abs_grads, i) for i in quantiles]

# fig, ax = plt.subplots(1, 1, figsize=(12, 6))
# ax.step(quantiles, grad_values, where="post")
# ax.set_xlabel("Quantile")
# ax.set_ylabel("Gradient Magnitude")
# plt.show()

### Generate datasets

We do this once then draw from it so that the random noise is the same each trial

In [11]:
FC_KEYS = list(FAULTY_CONDITIONS.keys())

for fc_key in FC_KEYS:
    log_header = f"updating/{fc_key}/data/"
    if not check_if_in_h5(logfilename, log_header+"df_data"):
        faulty_condition = FAULTY_CONDITIONS[fc_key]
        df = pd.read_csv(faulty_condition["path"], names=COLUMNS)
        df_data = default_rng.standard_normal(df.shape) * MEASUREMENT_NOISE_LEVEL/3 * df.values.mean(axis=0) + df.values
        logger.log_value(log_header+"df_data", df_data)

### Control Predictions

In [12]:
m = Exogenous_RkRNN(state_dim=len(X_COLUMNS), input_dim=len(U_COLUMNS), hidden_size=64)
m.load_state_dict(torch.load(rk_trained_weights_path, map_location=device, weights_only=True))

FC_KEYS = list(FAULTY_CONDITIONS.keys())
for fc_key in FC_KEYS:
    faulty_condition = FAULTY_CONDITIONS[fc_key]
    log_header = f"updating/{fc_key}/control/"
    if not check_if_in_h5(logfilename, log_header):
        data = logger.get_dataset(f"updating/{fc_key}/data/df_data")[0]
        df = pd.DataFrame(data, columns=COLUMNS)
        faulty_dataset = FCUDataset(df,
            X_scaler=X_scaler,
            U_scaler=U_scaler)
        faulty_dataloader = DataLoader(faulty_dataset, batch_size=128, shuffle=False, generator=torch.Generator(device=device))

        # try without updating the weights
        m.eval()
        faulty_data = flatten_predictions(m, faulty_dataloader)
        
        # mae and mse between each prediction and the actual values
        X0, U, Y, YP = faulty_data
        mae = np.abs(Y-YP).mean(axis=-1).mean(axis=-1)
        mse = ((Y-YP)**2).mean(axis=-1).mean(axis=-1)
        # scale
        Y_scaled = X_scaler.transform(Y.reshape(-1, len(X_COLUMNS))).reshape(Y.shape)
        YP_scaled = X_scaler.transform(YP.reshape(-1, len(X_COLUMNS))).reshape(YP.shape)
        mae_scaled = np.abs(Y_scaled-YP_scaled).mean(axis=-1).mean(axis=-1)
        mse_scaled = ((Y_scaled-YP_scaled)**2).mean(axis=-1).mean(axis=-1)
        
        logger = h5_logger(os.path.join(results_dir,"Faulty Conditions", "FC_GEKF.h5"), existing=True)
        logger.log_dict({
            f"{log_header}/X0": X0,
            f"{log_header}/U": U,
            f"{log_header}/Y": Y,
            f"{log_header}/YP": YP,
            f"{log_header}/mae": mae,
            f"{log_header}/mse": mse,
            f"{log_header}/mae_scaled": mae_scaled,
            f"{log_header}/mse_scaled": mse_scaled
        })

### KF Updates

In [13]:

fc_keys = FAULTY_CONDITIONS.keys()


for fc_key in fc_keys:
    trial_args = [
        {"log_header": f"updating/{fc_key}/full/"},
        {"log_header": f"updating/{fc_key}/t-0.99/", "thresh": grad_values[-2]}, 
        {"log_header": f"updating/{fc_key}/t-0.95/", "thresh": grad_values[-6]}, 
        {"log_header": f"updating/{fc_key}/q-0.99/", "quantile_thresh": 0.99},
        {"log_header": f"updating/{fc_key}/q-0.95/", "quantile_thresh": 0.95},
    ]
    for args in trial_args:
        sefk_updating_trial(fc_key, **args)

### ADAM UPDATES

In [15]:
LR = 3e-4
loss_threshold = 0.18

fc_keys = FAULTY_CONDITIONS.keys()

for fc_key in fc_keys:
    trial_args = [
        {"log_header": f"retraining/{fc_key}/full/"},  # all converged
        {"log_header": f"retraining/{fc_key}/t-0.99/", "thresh": grad_values[-2]}, 
        {"log_header": f"retraining/{fc_key}/t-0.95/", "thresh": grad_values[-6]}, 
        {"log_header": f"retraining/{fc_key}/q-0.99/", "quantile_thresh": 0.99},
        {"log_header": f"retraining/{fc_key}/q-0.95/", "quantile_thresh": 0.95},  # all converged
    ]
    for args in trial_args:
        retraining_trial(fc_key,
            lr=LR, loss_threshold=loss_threshold, early_stopping_threshold=0.15, divergence_threshold=20.0,
            moving_horizon_length=20, retraining_epochs=5,
            **args)

In [ ]:
retraining_df = get_retraining_stats(logger)
retraining_df

,faulty condition,full,t-0.99,t-0.95,q-0.99,q-0.95
0,Heat Exchanger Fouling,0.176112,0.180545,0.179543,0.185311,0.181208
1,Decreased Condenser Efficiency,0.170402,0.171087,0.170971,0.171120,0.170771
2,Heavy Naptha Flow Sensor Drift,0.169325,0.170602,0.170382,0.170539,0.170601
3,Higher Pressure Drop,0.299892,0.384869,0.377844,0.418643,0.400323
4,CAB Valve Leak,0.195650,0.220223,0.217721,0.232100,0.224943


In [17]:
updating_df = get_updating_stats(logger)
updating_df

,faulty condition,full,t-0.99,t-0.95,q-0.99,q-0.95
0,Heat Exchanger Fouling,0.168328,0.174741,0.172824,0.176612,0.173977
1,Decreased Condenser Efficiency,0.164993,0.169433,0.168131,0.168998,0.167801
2,Heavy Naptha Flow Sensor Drift,0.164849,0.168257,0.167596,0.168191,0.167368
3,Higher Pressure Drop,0.725912,0.597922,0.783820,0.353705,0.416279
4,CAB Valve Leak,0.210221,0.253780,0.271964,0.210080,0.314612


In [ ]:
param_selection_methods = ["full", "t-0.99", "t-0.95", "q-0.99", "q-0.95"]
retraining_less = retraining_df[param_selection_methods].values < updating_df[param_selection_methods].values
for fc_key in list(fc_keys)[:-2]:
    trial_args = [
        {"log_header": f"updating/{fc_key}/full/"},
        {"log_header": f"updating/{fc_key}/t-0.99/", "thresh": grad_values[-2]}, 
        {"log_header": f"updating/{fc_key}/t-0.95/", "thresh": grad_values[-6]}, 
        {"log_header": f"updating/{fc_key}/q-0.99/", "quantile_thresh": 0.99},
        {"log_header": f"updating/{fc_key}/q-0.95/", "quantile_thresh": 0.95},
    ]
    for args in trial_args:
        sefk_updating_trial(fc_key, find_unique_header= True, lr=2e-6, **args)
# for fc_key in list(fc_keys)[-1:]:
#     trial_args = [
#     #     {"log_header": f"updating/{fc_key}/full/"},
#     #     {"log_header": f"updating/{fc_key}/t-0.99/", "thresh": grad_values[-2]}, 
#     #     {"log_header": f"updating/{fc_key}/t-0.95/", "thresh": grad_values[-6]}, 
#     #     {"log_header": f"updating/{fc_key}/q-0.99/", "quantile_thresh": 0.99},
#         {"log_header": f"updating/{fc_key}/q-0.95/", "quantile_thresh": 0.95},
#     ]
#     for args in trial_args:
#         sefk_updating_trial(fc_key, find_unique_header= True, lr=2e-6, **args)

updating/Heat Exchanger Fouling/full_1/:   0%|          | 0/1378 [00:00<?, ?it/s]

### graph results

- error over time
- predictions
- weight evolution



In [ ]:
for key in FC_KEYS:
    base_path = os.path.join(results_dir,"Faulty Conditions", key)
    base_key = f"updating/{key}/"
    
    control_group = base_key+"control/"
    full_group = base_key+"full/"
    threshold_group = base_key+"threshold/"
    quantile_group = base_key+"quantile0.99/"
    
    ## Ensure that mae, mae_scaled, mse, mse_scaled are calculated for each group
    # for group in [full_group, threshold_group, quantile_group]:
    #     print(f"group: {group}")
    #     # for error in ["mae", "mae_scaled", "mse", "mse_scaled"]:
    #     #     logger.rm_key(group+error)
    #     X1 = logger.get_dataset(group+"X1")
    #     XP = logger.get_dataset(group+"XP")
        
    #     X1_rescaled = X_scaler.inverse_transform(X1.reshape(-1, len(X_COLUMNS))).reshape(X1.shape)
    #     XP_rescaled = X_scaler.inverse_transform(XP.reshape(-1, len(X_COLUMNS))).reshape(XP.shape)
        
    #     # mae
    #     if not check_if_in_h5(logfilename, group+"mae"):
    #         mae = np.abs(X1_rescaled-XP_rescaled).mean(axis=-1).mean(axis=-1)
    #         logger.log_dict({
    #             group+"mae": mae
    #         })
        
    #     # mae_scaled
    #     if not check_if_in_h5(logfilename, group+"mae_scaled"):
            
    #         mae_scaled = np.abs(X1-XP).mean(axis=-1).mean(axis=-1)
    #         logger.log_dict({
    #             group+"mae_scaled": mae_scaled
    #         })
        
    #     # mse
    #     if not check_if_in_h5(logfilename, group+"mse"):
    #         mse = ((X1_rescaled-XP_rescaled)**2).mean(axis=-1).mean(axis=-1)
    #         logger.log_dict({
    #             group+"mse": mse
    #         })
        
    #     # mse_scaled
    #     if not check_if_in_h5(logfilename, group+"mse_scaled"):
    #         mse_scaled = ((X1-XP)**2).mean(axis=-1).mean(axis=-1)
    #         logger.log_dict({
    #             group+"mse_scaled": mse_scaled
    #         })
        


In [ ]:
logger.get_group_keys("updating/CAB Valve Leak/full/")
logger.get_dataset("updating/CAB Valve Leak/full/parameters")

In [ ]:
faulty_condition = "Higher Pressure Drop"
# pressure: reactor
# pressure: delta
p_delta_idx = X_COLUMNS_NAMES.index("P.delta")
p_reactor_idx = X_COLUMNS_NAMES.index("P.reactor")
selected_indices = [p_delta_idx, p_reactor_idx]

true_data = np.concatenate([
    logger.get_dataset("updating/"+faulty_condition+"/control/Y")[0,:,0,[selected_indices]][0].T,
    logger.get_dataset("updating/"+faulty_condition+"/control/Y")[0,-1,1:,[selected_indices]][0].T
]).T

control_t10 = logger.get_dataset("updating/"+faulty_condition+"/control/YP")[0,:,9,[selected_indices]][0]
control_t60 = logger.get_dataset("updating/"+faulty_condition+"/control/YP")[0,:,-1,[selected_indices]][0]

kf_10 = X_scaler.inverse_transform(logger.get_dataset("updating/"+faulty_condition+"/full/YP")[:,0,9,:]).T[selected_indices]
kf_60 = X_scaler.inverse_transform(logger.get_dataset("updating/"+faulty_condition+"/full/YP")[:,0,-1,:]).T[selected_indices]


fig_hours = np.arange(true_data.shape[1])/60

fig, axs = plt.subplots(2,1, figsize=(13.3, 6), sharex=True, dpi=300)
axs[0].plot(fig_hours, true_data[0], color=true_colors[0], linewidth=3)
# axs[0].plot(np.arange(len(control_t10[0]))/60+10/60, control_t10[0], color=model_colors[0])
# axs[0].plot(np.arange(len(control_t60[0]))/60+1, control_t60[0], color=model_colors[2], linestyle="-")
# axs[0].plot(np.arange(len(kf_10[0]))/60+10/60, kf_10[0], color=kalman_colors[0])
# axs[0].plot(np.arange(len(kf_60[0]))/60+1, kf_10[0], color=kalman_colors[2], linestyle="-")
axs[0].set_title(r"Pressure: Delta (psi$_g$)")
axs[0].yaxis.set_major_locator(plt.MaxNLocator(3))
axs[0].set_xlim(-0.5, 24.5)
true, = axs[1].plot(fig_hours, true_data[1], color=true_colors[0], linewidth=3)
# original_t10, = axs[1].plot(np.arange(len(control_t10[1]))/60+10/60, control_t10[1], color=model_colors[0])
# original_t60, = axs[1].plot(np.arange(len(control_t60[1]))/60+1, control_t60[1], color=model_colors[2], linestyle="-")
# kf_t10, = axs[1].plot(np.arange(len(kf_10[1]))/60+10/60, kf_10[1], color=kalman_colors[0])
# kf_t60, = axs[1].plot(np.arange(len(kf_60[1]))/60+1, kf_10[1], color=kalman_colors[2], linestyle="-")
axs[1].set_title(r"Pressure: Reactor (psi$_g$)")
axs[1].yaxis.set_major_locator(plt.MaxNLocator(3))
axs[1].set_xlim(-0.5, 24.5)
axs[1].set_xlabel("Time (hours)")
axs[1].set_xticks([0, 4, 8, 12, 16, 20, 24])
axs[1].set_xticks(range(25), minor=True)
# axs[1].legend(
#     [true, original_t10, original_t60, kf_t10, kf_t60],
#     ["True", r"Original$_{t+10}$", r"Original$_{t+60}$", r"KF$_{t+10}$", r"KF$_{t+60}$"], loc="upper center",
#     bbox_to_anchor=(0.5, -0.36), ncol=5)
axs[1].legend([true], ["True"], bbox_to_anchor=(0.5, -0.36), ncol=1, loc="upper center")
fig.tight_layout()
fig.savefig(os.path.join(results_dir, "Faulty Conditions", "Higher Pressure Drop", "Pressure Drop Blank.png"), bbox_inches="tight", dpi=300)
fig.savefig(os.path.join(results_dir, "Faulty Conditions", "Higher Pressure Drop", "Pressure Drop Blank.svg"), bbox_inches="tight", dpi=300)
fig.savefig(os.path.join(results_dir, "Faulty Conditions", "Higher Pressure Drop", "Pressure Drop Blank.pdf"), bbox_inches="tight", dpi=300)

control_error = logger.get_dataset("updating/"+faulty_condition+"/control/mse_scaled")[0]
kf_full_error = logger.get_dataset("updating/"+faulty_condition+"/full/loss")
kf_q95_error = logger.get_dataset("updating/"+faulty_condition+"/q-0.95/loss")

fig, axs = plt.subplots(2,1, figsize=(13.3, 6), sharex=True, dpi=300)
axs[0].plot(fig_hours, true_data[0], color=true_colors[0], linewidth=3)
axs[0].plot(np.arange(len(control_t10[0]))/60+10/60, control_t10[0], color=model_colors[0])
axs[0].plot(np.arange(len(control_t60[0]))/60+1, control_t60[0], color=model_colors[2], linestyle="-")
axs[0].plot(np.arange(len(kf_10[0]))/60+10/60, kf_10[0], color=kalman_colors[0])
axs[0].plot(np.arange(len(kf_60[0]))/60+1, kf_10[0], color=kalman_colors[2], linestyle="-")
axs[0].set_title(r"Pressure: Delta (psi$_g$)")
axs[0].yaxis.set_major_locator(plt.MaxNLocator(3))
axs[0].set_xlim(-0.5, 24.5)
true, = axs[1].plot(fig_hours, true_data[1], color=true_colors[0], linewidth=3)
original_t10, = axs[1].plot(np.arange(len(control_t10[1]))/60+10/60, control_t10[1], color=model_colors[0])
original_t60, = axs[1].plot(np.arange(len(control_t60[1]))/60+1, control_t60[1], color=model_colors[2], linestyle="-")
kf_t10, = axs[1].plot(np.arange(len(kf_10[1]))/60+10/60, kf_10[1], color=kalman_colors[0])
kf_t60, = axs[1].plot(np.arange(len(kf_60[1]))/60+1, kf_10[1], color=kalman_colors[2], linestyle="-")
axs[1].set_title(r"Pressure: Reactor (psi$_g$)")
axs[1].yaxis.set_major_locator(plt.MaxNLocator(3))
axs[1].set_xlim(-0.5, 24.5)
# axs[2].plot(np.arange(len(control_error))/60, control_error, color=model_colors[0])
# axs[2].plot(np.arange(len(kf_full_error))/60, kf_full_error, color=kalman_colors[0])
# # axs[2].plot(np.arange(len(kf_q95_error))/60, kf_q95_error, color=kalman_colors[2])
# axs[2].set_title("Overall Prediction Mean Squared Error")
axs[1].set_xlabel("Time (hours)")
axs[1].set_xticks([0, 4, 8, 12, 16, 20, 24])
axs[1].set_xticks(range(25), minor=True)
axs[1].yaxis.set_major_locator(plt.MaxNLocator(3))
axs[1].legend(
    [true, original_t10, original_t60, kf_t10, kf_t60],
    ["True", r"Original$_{t+10}$", r"Original$_{t+60}$", r"KF$_{t+10}$", r"KF$_{t+60}$"], loc="upper center",
    bbox_to_anchor=(0.5, -0.57), ncol=5)
fig.tight_layout()
fig.savefig(os.path.join(results_dir, "Faulty Conditions", "Higher Pressure Drop", "Pressure Drop.png"), bbox_inches="tight", dpi=300)
fig.savefig(os.path.join(results_dir, "Faulty Conditions", "Higher Pressure Drop", "Pressure Drop.svg"), bbox_inches="tight", dpi=300)
fig.savefig(os.path.join(results_dir, "Faulty Conditions", "Higher Pressure Drop", "Pressure Drop.pdf"), bbox_inches="tight", dpi=300)

In [ ]:
faulty_condition = "Higher Pressure Drop"

kf_full_time = np.diff(logger.get_dataset("updating/"+faulty_condition+"/full-v2/time"), axis=0, prepend=0)
kf_q95_time = np.diff(logger.get_dataset("updating/"+faulty_condition+"/q-0.95/time"), axis=0, prepend=0)
# kf_q98_time = np.diff(logger.get_dataset("updating/"+faulty_condition+"/q-0.98/time"), axis=0, prepend=0)
kf_q99_time = np.diff(logger.get_dataset("updating/"+faulty_condition+"/q-0.99/time"), axis=0, prepend=0)
kf_t95_time = np.diff(logger.get_dataset("updating/"+faulty_condition+"/t-0.95-v2/time"), axis=0, prepend=0)
# kf_t98_time = np.diff(logger.get_dataset("updating/"+faulty_condition+"/t-q98/time"), axis=0, prepend=0)
kf_t99_time = np.diff(logger.get_dataset("updating/"+faulty_condition+"/t-0.99-v2/time"), axis=0, prepend=0)

kf_full_loss = logger.get_dataset("updating/"+faulty_condition+"/full-v2/loss")
kf_q95_loss = logger.get_dataset("updating/"+faulty_condition+"/q-0.95/loss")
# kf_q98_loss = logger.get_dataset("updating/"+faulty_condition+"/quantile0.98/loss")
kf_q99_loss = logger.get_dataset("updating/"+faulty_condition+"/q-0.99/loss")
kf_t95_loss = logger.get_dataset("updating/"+faulty_condition+"/t-0.95-v2/loss")
# kf_t98_loss = logger.get_dataset("updating/"+faulty_condition+"/threshol-q98/loss")
kf_t99_loss = logger.get_dataset("updating/"+faulty_condition+"/t-0.99-v2/loss")

fig, ax = plt.subplots(1, 1, figsize=(6,6), dpi=300)
plot_ci(kf_full_loss, kf_full_time, ax, label="All", color=kalman_colors[0])
plot_ci(kf_q99_loss, kf_q99_time, ax, label="Quantile 0.99", color=kalman_colors[0], marker_fmt="v")
plot_ci(kf_q95_loss, kf_q95_time, ax, label="Quantile 0.95", color=kalman_colors[1], marker_fmt="^")
# plot_ci(kf_q98_time, kf_q98_loss, ax, label="Quantile 0.98", color=kalman_colors[2], marker_fmt="v")
plot_ci(kf_t99_loss, kf_t99_time, ax, label="Threshold 0.99", color=kalman_colors[0], marker_fmt="s")
plot_ci(kf_t95_loss, kf_t95_time, ax, label="Threshold 0.95", color=kalman_colors[1], marker_fmt="D")
# plot_ci(kf_t98_loss, kf_t98_time, ax, label="Threshold (q98)", color=adam_colors[1])
ax.set_xlabel("Mean Squared Error")
ax.set_ylabel("Mean Time Per Iteration (s)")

legend_markers = [
    Line2D([0], [0], color=kalman_colors[0], marker="o", linestyle="None"),
    Line2D([0], [0], color=kalman_colors[0], marker="v", linestyle="None"),
    Line2D([0], [0], color=kalman_colors[1], marker="^", linestyle="None"),
    # Line2D([0], [0], color=kalman_colors[2], marker="v", linestyle="None"),
    Line2D([0], [0], color=kalman_colors[0], marker="s", linestyle="None"),
    Line2D([0], [0], color=kalman_colors[1], marker="D", linestyle="None"),
]
legend_labels = [
    "All",
    "Quantile 0.99",
    "Quantile 0.95",
    # "Quantile 0.98",
    "Threshold 0.99",
    "Threshold 0.95",
]

ax.legend(legend_markers, legend_labels)

fig.savefig(os.path.join(results_dir, "Faulty Conditions", "Higher Pressure Drop", "Time vs Loss.png"), bbox_inches="tight", dpi=300)
fig.savefig(os.path.join(results_dir, "Faulty Conditions", "Higher Pressure Drop", "Time vs Loss.svg"), bbox_inches="tight", dpi=300)


heavy naptha flow

In [ ]:
df_stable = pd.read_csv(NOC_STABLE_PATH, names=COLUMNS)
df_faulty = pd.read_csv(FAULTY_NAPTHASENSOR_PATH, names=COLUMNS)
df = pd.concat([df_stable, df_faulty], ignore_index=True)
data = default_rng.standard_normal(df.shape) * MEASUREMENT_NOISE_LEVEL/3 * df.values.mean(axis=0) + df.values
df = pd.DataFrame(data, columns=COLUMNS)

HN_idx = X_COLUMNS_NAMES.index("F.HN")

kf_30 = X_scaler.inverse_transform(logger.get_dataset("updating/"+"Heavy Naptha Flow Sensor Drift"+"/full/XP")[:,0,30,:]).T[HN_idx]
control_t30 = logger.get_dataset("updating/"+faulty_condition+"/control/YP")[0,:,30,[HN_idx]][0]

fig, ax = plt.subplots(1, 1, figsize=(6,6), dpi=300)

ax.plot(np.arange(df.shape[0])/60, df["F.HN"], color=true_colors[0], label="Measured Data")
ax.plot(np.arange(kf_30.shape[0])/60+30/60+48, kf_30, color=kalman_colors[0], linewidth=1.5, label=r"KF-Updated$_t+30$")
ax.plot(np.arange(control_t30.shape[0])/60+30/60+48, control_t30, color=model_colors[0], linewidth=1.5, label=r"Original Model$_t+30$")

ax.vlines(48, min(df["F.HN"])-10, max(df["F.HN"])+10, color="black", linestyle="--", linewidth=1)
ax.text(24, max(df["F.HN"]), "Normal Operating\nConditions", ha="center")
ax.text(60, max(df["F.HN"]), "Sensor\nFault", ha="center")
ax.set_xlabel("Time (hours)")
ax.set_xticks([0, 12, 24, 36, 48, 60, 72])
ax.set_xticks(np.arange(72, step=4), minor=True)
ax.legend(bbox_to_anchor=(0.5, -0.2), ncol=3)

In [ ]:

fig, ax = plt.subplots(1, 1, figsize=(13,4.5), dpi=300)

ax.plot(np.arange(df.shape[0])/60, df["F.HN"], color=true_colors[0], label="Measured Data")
ax.plot(np.arange(kf_30.shape[0])/60+30/60+48, kf_30, color=kalman_colors[0], linewidth=1.5, label=r"KF-Updated$_{t+30}$")
ax.plot(np.arange(control_t30.shape[0])/60+30/60+48, control_t30, color=model_colors[0], linewidth=1.5, label=r"Original Model$_{t+30}$")

ax.vlines(48, min(df["F.HN"])-10, max(df["F.HN"])+10, color="black", linestyle="--", linewidth=1)
ax.text(24, max(df["F.HN"]), "Normal Operating\nConditions", ha="center")
ax.text(60, max(df["F.HN"]), "Sensor\nFault", ha="center")
ax.set_xlabel("Time (hours)")
ax.set_xticks([0, 12, 24, 36, 48, 60, 72])
ax.set_xticks(np.arange(72, step=4), minor=True)
ax.set_ylabel("Heavy Naptha Flow (lb/min)")
ax.legend(bbox_to_anchor=(0.5, -0.12), ncol=3, loc="upper center")

fig.savefig(os.path.join(results_dir, "Faulty Conditions", "Heavy Naptha Flow Sensor Drift", "Heavy Naptha Flow.png"), bbox_inches="tight", dpi=300)
fig.savefig(os.path.join(results_dir, "Faulty Conditions", "Heavy Naptha Flow Sensor Drift", "Heavy Naptha Flow.svg"), bbox_inches="tight", dpi=300)

In [ ]:
# logger = h5_logger(os.path.join(results_dir,"Faulty Conditions", "FC_GEKF.h5"), existing=True)

# for key in FC_KEYS:
#     base_path = os.path.join(results_dir,"Faulty Conditions", key)
#     base_key = f"updating/{key}/"
    
#     control_group = base_key+"control/"
#     full_group = base_key+"full/"
#     threshold_group = base_key+"threshold/"
#     quantile_group = base_key+"quantile0.99/"
    
#     errors = {
#         "mae": "Mean Absolute Error",
#         "mae_scaled": "Mean Absolute Error (Scaled)",
#         "mse": "Mean Squared Error",
#         "mse_scaled": "Mean Squared Error (Scaled)"
#     }
    
#     # graph error over time
#     for error, name in errors.items():
#         fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=300)
#         ax.plot(logger.get_dataset(control_group+error)[0], "k-", label="No Updates")
#         ax.plot(logger.get_dataset(full_group+error)[0], label="Update All Parameters")
#         ax.plot(logger.get_dataset(threshold_group+error)[0], label=fr"Update Parameters $|\nabla L| >$ {thresh:.4f}")
#         ax.plot(logger.get_dataset(quantile_group+error)[0], label=r"Update Parameters $|\nabla L| >$ 0.99 Quantile")
        
#         ax.set_xlabel("Prediction Time")
#         ax.set_xticks(np.arange(0, 1441, 120))
#         ax.set_xlim(0, 1440)
#         ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{x/60:.0f}:00"))
#         ax.set_ylabel(name)
#         ax.set_title(f"{key}: Error Over Time")
#         ax.legend(loc="upper center", bbox_to_anchor=(0.5, -0.075), ncol=2)
        
#         fig.savefig(os.path.join(base_path, f"{error}.png"), bbox_inches="tight")
#         fig.savefig(os.path.join(base_path, f"{error}.svg"), bbox_inches="tight")
#         plt.close(fig)

In [ ]:
# index = 600
# rescale = True
# scale = 1
# figsize = (32*scale, 16*scale)

# for key in FC_KEYS:
#     base_path = os.path.join(results_dir,"Faulty Conditions", key)
#     base_key = f"updating/{key}/"
    
#     control_group = base_key+"control/"
#     full_group = base_key+"full/"
#     threshold_group = base_key+"threshold/"
#     quantile_group = base_key+"quantile0.99/"
    
#     x0 = logger.get_dataset(control_group+"X0").squeeze()[index]
#     u_vars = logger.get_dataset(control_group+"U").squeeze()[index]
#     true = logger.get_dataset(control_group+"Y").squeeze()[index]
#     control = logger.get_dataset(control_group+"YP").squeeze()[index]
#     full = logger.get_dataset(full_group+"XP").squeeze()[index]
#     threshold = logger.get_dataset(threshold_group+"XP").squeeze()[index]
#     quantile = logger.get_dataset(quantile_group+"XP").squeeze()[index]
    
#     if rescale:
#         full = X_scaler.inverse_transform(full.reshape(-1, len(X_COLUMNS))).reshape(full.shape)
#         threshold = X_scaler.inverse_transform(threshold.reshape(-1, len(X_COLUMNS))).reshape(threshold.shape)
#         quantile = X_scaler.inverse_transform(quantile.reshape(-1, len(X_COLUMNS))).reshape(quantile.shape)
#     else:
#         control = X_scaler.transform(control.reshape(-1, len(X_COLUMNS))).reshape(control.shape)
#         x0 = X_scaler.transform(x0.reshape(-1, len(X_COLUMNS))).reshape(x0.shape)
#         u_vars = U_scaler.transform(u_vars.reshape(-1, len(U_COLUMNS))).reshape(u_vars.shape)
#         true = X_scaler.transform(true.reshape(-1, len(X_COLUMNS))).reshape(true.shape)
        
#     x_prefix = "" if rescale else "Scaled "
#     u_prefix = "" if rescale else "Scaled "
#     units = "" if rescale else "Scaled "
    
#     with plt.rc_context({"xtick.labelsize": 16, "ytick.labelsize": 16, "axes.labelsize": 16, "axes.titlesize": 16}):
#         fig, axs = plt.subplots(12, 4, figsize=figsize)
#         for i, ax in enumerate(axs.flatten()):
#             x_coords = np.arange(true.shape[0])+1
#             # X
#             if i < 31:
#                 # X
#                 label = sorted(X_COLUMNS_NAMES)[i]
#                 label_index = X_COLUMNS_NAMES.index(label)
#                 ax.plot(x_coords, true[:, label_index], "k-")
#                 ax.scatter(0, x0[label_index], c="k", marker="o")
#                 ax.plot(x_coords, control[:, label_index])
#                 ax.plot(x_coords, full[:, label_index])
#                 ax.plot(x_coords, threshold[:, label_index])
#                 ax.plot(x_coords, quantile[:, label_index])
#                 ax.set_title(label)
                
#             elif (i > 31) and (i < 46):
#                 # U
#                 ii = i - 32
#                 label = sorted(U_COLUMNS_NAMES)[ii]
#                 label_index = U_COLUMNS_NAMES.index(label)
#                 ax.plot(u_vars[:, label_index], "k-")
#                 ax.set_title(label)
            
#             else:
#                 ax.axis('off')
#                 continue
            
#         fig.text(-0.00, 8/12, ha="center", va="center", rotation=90, s=f"{x_prefix}Process Variables", fontsize=16, fontweight="bold")
#         fig.text(-0.00, 2/12, ha="center", va="center", rotation=90, s=f"{u_prefix}Exogenous Inputs", fontsize=16, fontweight="bold")
#         fig.text(0.5, 0, ha="center", va="center", s="Prediction Horizon", fontsize=16, fontweight="bold")
#         fig.text(0.5, 0.7/12, ha="left", va="center", s=f"Prediction index: {index} of {logger.get_dataset(control_group+'Y').squeeze().shape[0]}", fontsize=14)
#         # fig.text(0.5, 0.5/12, ha="left", va="center", s=f"Prediction {units}MAE: {nMAE:4g}", fontsize=14)

#         fig.tight_layout()
#         fig.patches.extend([plt.Rectangle(xy=(-0.01,-0.2/12), width=1.01, height=4.2/12, fill=True, color="lightgrey", zorder=-1, transform=fig.transFigure, figure=fig)])
            
#         # fig.subplots_adjust(hspace=0.9)
#         fig.text(0.5, 1.00, f"{key}: Prediction Comparison", ha="center", va="center", fontsize=20, fontweight="bold")
#         fig.legend(["True", "Initial Value", "No Updates", "Full Updates", "Threshold Updates", "Quantile Updates"], bbox_to_anchor=(3.1/4,5/12), loc="upper left", ncol=2)
#         fig.tight_layout()
    
    
    